# Control Systems 1, NB07: Time-Domain specifications and PID

© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the steady-state error, different Time-Domain Specifications for first and second order systems. It also features how to approximate higher order system using Dominant pole approximation. Furthermore, it introduces the PID-controller.

Authors:
- Jonas Holinger; jholinger@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

# Learning Objectives


After completing this notebook, you should be able to:

1. Determine the steady-state error of a system.
2. Understand different time-domain specifications for first and second order system.
3. Understand how to design the system such that the specifications are met.
4. Understand how to approximate higher order systems using dominant pole approximation
5. Have a basic understanding of PID-Controllers.

In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

In [ ]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from scipy.integrate import odeint
from IPython.display import display, clear_output, Math
from ipywidgets import interactive, FloatSlider, VBox, ToggleButton, FloatSlider
from scipy import signal
import warnings
warnings.filterwarnings("ignore")

# Steady State Error

Previously we have looked at how to determine the time response of a system. Building upon that, typically we wish for the system to approach some desired value (i.e., the reference). Consider that for the cruise control system of a vehicle, the vehicle should approach and remain at the preset value. However, depending on the type of system, it is not always the case that the time response will approach exactly the reference value; Instead resulting in an offset no matter how long you continue running the system for. This offset is known as the steady-state error, and is the difference between the systems' reference signal $r(t)$ and the closed-loop output $y(t)$ as $\lim_{t \to \infty}$.

One way of determining whether a system will have a steady-state error is by applying the final value theorem:
$$ e_{ss}=  \displaystyle \lim_{t \to \infty } e(t) =  \displaystyle \lim_{s \to 0} s \cdot E(s) $$


### Example


Suppose we have the following open-loop transfer function $L(s)$:
$$ L(s) = C(s)P(s) = K\frac{1}{s}\frac{1}{s(s+2)} $$

Furthermore, suppose that the input to the system is a first-order ramp ($q=1$) such that $r(t) = t$. This input has the corresponding laplace transform:
$$ R(s) = \frac{1}{s^2} $$

To calculate the error signal $E(s)$, we use the sensitivity function, which is known from previous lectures. The sensitivity function maps the reference input $r$ to the error $e$:
$$ S(s) = \frac{E(s)}{R(s)} = \frac{1}{1+P(s)C(s)} = \frac{1}{1+L(s)} $$

We can multiply the sensitivity function $S(s)$ with our input $R(s)$ and apply the final value theorem to get our steady-state error:
$$ e_{ss} = \displaystyle \lim_{s \to 0} s \cdot E(s) = \displaystyle \lim_{s \to 0} s \cdot R(s) \cdot \frac{1}{1+L(s)} = \displaystyle \lim_{s \to 0} s \cdot \frac{\frac{1}{s^2}}{1+\frac{K}{s(s+2)}} $$

The steady-state error we get after applying the limit is
$$ e_{ss} = \frac{2}{K} $$

> In fact, it can be derived, that by rearranging the transfer function into bode form, if the system is stable and $q=0$, the Bode gain corresponds to the steady-state gain for a unit step input and is called the DC gain.

If we now add a second integrator to our system, such that $L(s) = K\frac{1}{s^2}\frac{1}{s(s+2)}$, and calculating the steady-state error again:
$$ e_{ss} = \displaystyle \lim_{s \to 0} s \cdot \frac{\frac{1}{s^2}}{1+\frac{K}{s^2(s+2)}} $$
we see that our steady-state error is exactly zero.


### Conclusion


To achieve a steady-state error of zero, you need to have one more integrator than the order of your ramp input. To achieve a small steady-state error, you can increase the Bode gain. To avoid calculating the steady-state error each time, you can use the following table:




| $ e_{ss} $ | $ q = 0 $                   | $ q = 1 $            | $ q = 2 $            |
|-------------|-----------------------------|----------------------|----------------------|
| Type 0      | $ \frac{1}{1 + K_{\text{Bode}}} $ | $ \infty $           | $ \infty $           |
| Type 1      | $ 0 $                        | $ \frac{1}{K_{\text{Bode}}} $ | $ \infty $           |
| Type 2      | $ 0 $                        | $ 0 $                | $ \frac{1}{K_{\text{Bode}}} $ |

### Visualization
Try different combinations of system types and input ramps and observe whether you have a steady-state error or not.


In [ ]:
def simulate_system(system_type, input_type):

    t = np.linspace(0, 125, 1000)

    P = ct.TransferFunction([1], [1, 1])

    if system_type == "Type 0":
        C = ct.TransferFunction([1], [1])
    elif system_type == "Type 1":
        C = ct.TransferFunction([1], [1, 0])
    elif system_type == "Type 2":
        C = ct.TransferFunction([1, 0], [1, 0, 0])

    open_loop_system = C * P
    closed_loop_system = ct.feedback(open_loop_system)

    if input_type == "Unit Ramp" and system_type == "Type 2":
        t = np.linspace(0, 1500, 1000)

    elif input_type == "Unit Parabola" and system_type == "Type 2":
        t = np.linspace(0, 300, 1000)


    if input_type == "Unit Step":
        input_signal = np.ones_like(t)
    elif input_type == "Unit Ramp":
        input_signal = t
    elif input_type == "Unit Parabola":
        input_signal = 0.5 * t**2


    t_out, y_out = ct.forced_response(closed_loop_system, T=t, U=input_signal)

    plt.figure(figsize=(12, 6))
    plt.plot(t, input_signal, 'r--', label='Input Signal', linewidth=2)
    plt.plot(t_out, y_out, 'b-', label='Output Response ', linewidth=2)



    plt.title(f'Input and Output of {system_type} System to {input_type} ')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()



system_type_dropdown = widgets.Dropdown(
    options=["Type 0", "Type 1", "Type 2"],
    value="Type 0",
    description='System Type:',
)

input_type_dropdown = widgets.Dropdown(
    options=["Unit Step", "Unit Ramp", "Unit Parabola"],
    value="Unit Step",
    description='Input Type:',
)


interactive_plot = interactive(simulate_system,
                               system_type=system_type_dropdown,
                               input_type=input_type_dropdown)
display(interactive_plot)


# Time Domain Specifications

Last week we saw that provided with an open-loop system, $L(s)$, how, using a feedback architecture, we can quickly analyze whether the closed-loop system design is feasible (e.g., stable) for some varying system parameter (e.g., the feedback gain) using the root locus. Although using this approach we could qualitatively infer whether there exists some value of $K$, such that the system is stable and/or that the time response would not oscillate/overshoot, in practice we wish to specify the *extent* to which a system can overshoot, or how *fast* the system should achieve its steady state value. In general, a system can have such specifications in the time-domain (i.e., on the time response) or in the frequency domain (we will start looking at the frequency domain starting next week). The below section is dedicated to describing the time-domain specifications for first, second and higher-order systems.

## First-Order Systems


Consider a stable first-order system with the transfer function: <br>$$G(s)=\frac{K}{τs+1}$$<br>
Recall that with an inital condition of $x(0)=0$, the time response is given by: $$y(t)=K-e^{\frac{-t}{\tau}}$$

Then, we are interested in the following time-domain specifications:

**Steady State/DC gain, $y_{ss}$,** is a measure of how much the systems output responds to an input after the transients effects have died out. In our first order system the DC gain corresponds to K.

**Time constant, $\tau$,** measures the speed of the system's response. It is defined as the time required for the system's unit step response to reach approximately 63% of its final value.

**Settling time, $T_d$,** is the time it takes for the output to remain within $d\%$ of the final value. The settling time can be approximated using the time constant $τ$ using the formula $$T_d=τ \cdot \ln {\frac{100}{d}}$$ For $5\%$ the approximation is $T_d=3\tau$

Note that:
- The $\log{(\cdot)}$ provided in the lectures are natural logarithms. 
- There are various equivalent forms of the settling time -- e.g., $T_d = - \frac{\ln{d\%}}{\sigma}$, where $\sigma$ is the real part of the poles of $G(s)$. 
- The time constant is given by $\tau = -\frac{1}{p}$, where $p$ is the pole of G(S).

### Visualization

Below we have plotted the unit step response of a first-order system. Similar to above, the first-order system is parametrized by $K$ and $\tau$.

Try:
- Varying $K$, what do you notice about the output response? 
- Varying $\tau$, what do you notice about the output response?

It is interesting to note that although the specifications themselves vary accordingly with the parameters, the behaviour of the output response remains unchanged (assuming an asymptotically stable system).

In [ ]:
def plot_first_order_system(K=3, tau=2):
    sys = ct.TransferFunction([K], [tau, 1])
    t = np.linspace(0, 10 * tau, 1000)
    t, y = ct.step_response(sys, T=t)

    tau_time = tau
    yss = K
    settling_time = -np.log(0.05) * tau

    y_tau = K * (1 - np.exp(-tau_time / tau))
    y_settling = K * (1 - np.exp(-settling_time / tau))

    plt.figure(figsize=(10, 6))
    plt.plot(t, y, label="Step Response", color="blue")
    plt.plot(t, np.ones_like(t), label="Step Input", color="orange", linestyle='--')
    plt.axhline(yss, color="red", linestyle="--")
    plt.plot([tau_time, tau_time], [0, y_tau], color="green", linestyle="--")
    plt.plot([settling_time, settling_time], [0, y_settling], color="purple", linestyle="--")
    plt.text(tau_time, 0, f"τ    ", color="green", ha="center", va="bottom", fontsize=10, fontweight="bold")
    plt.text(settling_time, 0, f"Settling Time    ", color="purple", ha="center", va="bottom", fontsize=10, fontweight="bold")
    plt.text(0, yss, f"DC Gain", color="red", ha="left", va="bottom", fontsize=10, fontweight="bold")

    plt.xlabel("Time (s)")
    plt.ylabel("Response")
    plt.title(f"Step Response of First-Order System: G(s) = {K:.2f}/({tau:.2f}s + 1)")
    plt.legend(loc="best")
    plt.grid(True)
    plt.ylim(0, 1.2 * yss)
    plt.show()

K_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="K:")
tau_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="τ:")

interactive_plot = interactive(plot_first_order_system, K=K_slider, tau=tau_slider)
display(interactive_plot)


## Second-Order System

### Time Response


Now, consider that the form of a second-order system is:
$$G(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}
\quad \Longleftrightarrow \quad
\begin{cases}
    \dot{x} = \begin{bmatrix} 0 & 1 \\ -\omega_n^2 & -2 \zeta \omega_n \end{bmatrix} x + \begin{bmatrix} 0 \\ \omega_n^2 \end{bmatrix} u \\
    y = x
\end{cases}
$$
We get the time reponse by imposing a zero initial condition and solving analytically:

$$y(t) = 1 - \frac{1}{\cos \varphi} e^{\sigma t} \cos(\omega t + \varphi), \quad t \geq 0.$$

where $\varphi=arctan(\frac{σ}{ω})$

Similar to as in the first-order system, there are two main parameters:
- $\omega_n$ denotes the natural frequency of our system, sometimes also called the eigenfrequency. The natural frequency is defined as $\omega_n^2=\sigma^2+\omega^2$ and can be calculated from the poles of the system $s=\sigma\pm j\omega$ 
- The damping factor $\zeta = \frac{\sigma}{\omega_n}$ can be used to categorize our system into the following cases:
  * $\zeta = 0$, undamped
  * $0 < \zeta<1$, underdamped
  * $\zeta=1$, critically damped
  * $\zeta>1$, overdamped


#### Visualization

To get an intuition as to how the system parameters affect the time-response, use the interactive plot below.
- How does the time response vary with $\zeta$?
- How does the time response vary with $\omega_n$?

In [ ]:
def plot_damping_responses(omega_n):
    damping_ratios = {
        "Underdamped (ζ < 1)": 0.5,
        "Critically Damped (ζ = 1)": 1.0,
        "Overdamped (ζ > 1)": 1.5
    }

    t = np.linspace(0, 10, 500)

    plt.figure(figsize=(12, 8))

    for label, zeta in damping_ratios.items():
        num = [omega_n**2]
        den = [1, 2*zeta*omega_n, omega_n**2]
        system = signal.TransferFunction(num, den)

        t, response = signal.step(system, T=t)

        plt.plot(t, response, label=label)

    plt.title('Step Responses of Second Order Systems')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.axhline(1, color='gray', linestyle='--', linewidth=0.8)
    plt.grid(True)
    plt.legend()
    plt.show()

omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')

interactive_plot = interactive(plot_damping_responses, omega_n=omega_n_slider)

display(interactive_plot)


### Time-Domain Specifications

Similar to the first-order system, the following time specifications are relevant, and whose definitions remain (mostly) the same.

- The **Steady-state Gain** remains the same with that for the first order system.
- The **Time Constant** is now determined using the exponential envelope of the response. It can be determined through: $τ=\frac{1}{{|\sigma|}}$
- The **Settling Time** within $d\%$ of steady-state is also the same as in the first order system: $T_d = \tau \ln{\frac{100}{d}}$
  - For an overdamped system, where the poles are real and distinct, the settling time $T_d$ should be calculated using the pole with the largest time constant $\tau$. This pole represents the slowest decay rate, which dominates the system's response.

However, since the second order system has a fundamentally different behavior when the system is underdamped (i.e., due to the overshoot), in such cases we are also interested in the following specifications:

- The **Time to peak, $T_p$,** is the duration it takes for the system response to reach its maximum value. It is given by:
$T_p=\frac{\pi}{\omega}$

  - **Hint:** To calculate $\omega$ given the natural frequency $w_n$ and damping coefficient $\zeta$ use: $\omega=\omega_n\sqrt{1-\zeta^2}$

- The **Peak Overshoot Ratio, $M_p$** is the maximum amount, by which the response exceeds the steady-state value. It can be expressed as: $M_p=e^{\frac{\sigma\pi}{\omega}}$. Alternatively, it can be related to the damping ratio $\zeta$ using: $\zeta^2=\frac{(\ln M_p)^2}{\pi^2+(\ln M_p)^2}$. 
  - Note that $M_p$ may be expressed as a percentage, e.g., $30\%$, but must be entered into the form above as its decimal equivalent, e.g., $0.3$.

- The **Rise Time, $T_{100}$,** is defined as the time required for the response to rise from 0% to 100% of the steady-state value. $T_{100\%}=\frac{\frac{\pi}{2}-\phi}{\omega}\approx \frac{\pi}{2\omega_n}$
  - Note that depending on the application, one may be interested in the other definitions of the rise time, e.g., from 0% to 90%, or from 10% to 90%. Within the course, if such a case arises, this will be made explicit.


#### Visualisation


You can adjust the damping ratio and the natural frequency of the systems with the sliders. By pressing the buttons, you can display the different time-domain specifications. Observe how they change when you adjust the system's parameters.

In [ ]:
#@title Run code

def plot_step_response(omega_n, zeta, show_time_to_peak, show_peak_overshoot, show_rise_time, show_settling_time, show_time_constant):

    num = [omega_n**2]
    den = [1, 2*zeta*omega_n, omega_n**2]
    system = signal.TransferFunction(num, den)


    t = np.linspace(0, 20, 500)
    t, response = signal.step(system, T=t)

    # Time to Peak
    Tp = np.pi / (omega_n * np.sqrt(1 - zeta**2)) if zeta < 1 else np.nan

    # Peak Overshoot Ratio
    OS = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2)) * 100 if zeta < 1 else np.nan

    # Rise Time
    omega_d = omega_n * math.sqrt(1 - zeta**2) if zeta != 1 else np.nan
    theta = math.acos(zeta)
    Tr= (math.pi - theta) / omega_d

    # Settling Time
    Ts = 4 / (zeta * omega_n) if zeta > 0 else np.nan

    # Time Constant
    tau = 1 / (zeta * omega_n) if zeta > 0 else np.nan

    plt.figure(figsize=(20, 6))

    plt.subplot(1, 2, 1)
    plt.plot(t, response, label='Step Response')

    if show_time_to_peak and not np.isnan(Tp):
        plt.text(Tp, 0, f'Tp  ', color='red', fontsize=10, ha='right')
        plt.plot([Tp, Tp], [0, response[np.where(t >= Tp)[0][0]]], color='r', linestyle='--')

    if show_peak_overshoot:
        plt.text(Tp, response.max(), f'OS', color='red', fontsize=10, ha='center', va='bottom')

    if show_rise_time and not np.isnan(Tr):
        plt.text(Tr,0, f'T100%   ', color='green', fontsize=10, ha='right')
        plt.plot([Tr, Tr], [0, response[np.where(t >= Tr)[0][0]]], color='g', linestyle='--')

    if show_settling_time and not np.isnan(Ts):
        plt.text(Ts, 0, f'Td ', color='magenta', fontsize=10, ha='right')
        plt.plot([Ts, Ts], [0, response[np.where(t >= Ts)[0][0]]], color='m', linestyle='--')

    if show_time_constant and not np.isnan(tau):
        plt.text(tau, 0, f'τ ', color='cyan', fontsize=10, ha='right')
        plt.plot([tau, tau], [0, response[np.where(t >= tau)[0][0]]], color='c', linestyle='--')

    plt.title('Step Response of Second Order System')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.legend()
    plt.grid(True)
    plt.ylim(0, 1.1 * response.max())

    poles = np.roots(den)

    # Pole Plot
    plt.subplot(1, 2, 2)
    plt.scatter(np.real(poles), np.imag(poles), color='blue', marker='x')
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.title('Poles of the System')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')
zeta_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, description='ζ')

show_time_to_peak_btn = ToggleButton(value=False, description='Time to Peak', button_style='info')
show_peak_overshoot_btn = ToggleButton(value=False, description='Peak Overshoot', button_style='info')
show_rise_time_btn = ToggleButton(value=False, description='Rise Time (T100%)', button_style='info')
show_settling_time_btn = ToggleButton(value=False, description='Settling Time', button_style='info')
show_time_constant_btn = ToggleButton(value=False, description='Time Constant', button_style='info')

interactive_plot = interactive(
    plot_step_response,
    omega_n=omega_n_slider,
    zeta=zeta_slider,
    show_time_to_peak=show_time_to_peak_btn,
    show_peak_overshoot=show_peak_overshoot_btn,
    show_rise_time=show_rise_time_btn,
    show_settling_time=show_settling_time_btn,
    show_time_constant=show_time_constant_btn
)

display(interactive_plot)


### Higher Order Systems - Dominant Pole Approximation

Systems higher than 2 dimensions are often more difficult to analyze. Hence, it is sometimes desirable to approximate a higher order system as a second (or even first) order system, and then apply the specifications to the approximation. 

Recall that the transfer function of a system can be rewritten in its partial fraction expansion, where each "fraction" is a direct component of the total output response. 

$$G(s) = \frac{r_1}{s-p_1} + \frac{r_2}{s-p_2} + \frac{r_3}{s-p_3} + \frac{r_4}{s-p_4} + \ldots\quad \Leftrightarrow
\quad g(t) = r_1 e^{p_1t} + r_2 e^{p_2t} + r_3 e^{p_3t} + r_4 e^{p_4t} + \ldots 
$$

By observing this form closely one can see that the system response is governed by the terms with larger values of real part of the poles -- i.e., if the system is stable, the pole closest to zero has a lowest decay rate, and thus will domainate the total response as the other transients tend to $0$. The reponse of the terms with more negative poles die out faster than the ones with larger pole values. Hence, **dominant poles** are those with the least negative real part. 
- It is important to note there can be exceptions to this rule. Consider the case where the residue is very small, then the poles contribution to the final response is very small, and hence this pole may not dominate the total response.

#### Visualization

To help provide an intuitive understanding to the extent which the dominant poles approximate the total response, input a transfer function by the numerator and denominator, and then how many dominant poles you wish to approximate it using. 

- Can you find a transfer function for which a first-order approximation is a suitable representation of the higher-order system?
- Can you find a transfer function. 

Note that we have provided a bode plot visualization as foresight, since these will be introduced in the following weeks. 

In [ ]:
import numpy as np
import sympy as sp
from scipy.signal import zpk2tf, residue, TransferFunction, bode, step, impulse
import ipywidgets as widgets
from IPython.display import display, clear_output, Math, Latex
import matplotlib.pyplot as plt

def parse_complex(s):
    """Parse a string into a complex number, handling 'i' as 'j'."""
    try:
        return complex(s.replace('i', 'j').replace(' ', ''))
    except:
        raise ValueError(f"Invalid complex number: {s}")

def dominant_pole_approximation(num, den, num_poles):
    """Perform dominant pole approximation by retaining specified number of poles."""
    # Find poles and zeros
    zeros = np.roots(num)
    poles = np.roots(den)

    # Sort poles by their real parts (smallest real part indicates dominant poles)
    poles_sorted = poles[np.argsort(np.abs(np.real(poles)))]
    dominant_poles = poles_sorted[:num_poles]

    # Reconstruct the approximated denominator polynomial
    den_approx = np.poly(dominant_poles)

    # For zeros, we'll keep all zeros for now (you might adjust this based on your approximation needs)
    num_approx = num.copy()

    # Adjust the gain to match the DC gain (or another frequency if desired)
    # Compute DC gain of original system
    w0 = 0  # DC frequency
    _, mag_orig, _ = bode(TransferFunction(num, den), [w0])
    _, mag_approx, _ = bode(TransferFunction(num_approx, den_approx), [w0])

    # Adjust the numerator to match the DC gain
    gain_adjustment = 10**((mag_orig[0] - mag_approx[0])/20)
    num_approx = gain_adjustment * num_approx

    return num_approx, den_approx

def display_transfer_function(num, den, zeros, poles, residues, poles_res, k_res, num_approx=None, den_approx=None, plot_type='Bode Plot', output_area=None):
    # """Display the transfer function in various forms and plot the selected response."""
    # with output_area:
        clear_output()  # Clear previous output in the output area
        display( num_coeffs_input, den_coeffs_input, num_poles_input, plot_type_widget, calc_button)
        s = sp.symbols('s', complex=True)

        # Rational Function Form
        num_poly = sp.Poly(num, s)
        den_poly = sp.Poly(den, s)
        H_s = num_poly.as_expr() / den_poly.as_expr()
        display(Latex('**Rational Function Form:**'))
        display(Math(r'H(s) = \frac{%s}{%s}' % (sp.latex(num_poly.as_expr()), sp.latex(den_poly.as_expr()))))

        # Factored Form
        zeros_expr = sp.Mul(*[s - z for z in zeros])
        poles_expr = sp.Mul(*[s - p for p in poles])
        gain = num[0] / den[0]
        display(Latex('**Factored Form:**'))
        display(Math(r'H(s) = %s \times \frac{%s}{%s}' % (sp.latex(gain), sp.latex(zeros_expr.evalf(2)), sp.latex(poles_expr.evalf(2)))))

        # Partial Fraction Expansion
        partial_frac_terms = [r'\frac{%s}{s - (%s)}' % (sp.latex(np.round(r,2)), sp.latex(np.round(p,2))) for r, p in zip(residues, poles_res)]
        partial_frac = ' + '.join(partial_frac_terms)
        if k_res.size > 0:
            k_poly = sp.Poly(k_res, s)
            k_expr = sp.latex(k_poly.as_expr())
            partial_frac += ' + ' + k_expr
        # display(Latex('**Partial Fraction Expansion:**'))
        # display(Math('H(s) = %s' % partial_frac))

        display(Latex(r'**Partial Fraction Expansion:**'))
        display(Math(r'H(s) = %s' % partial_frac))


        # Plotting
        display(Latex(f'**{plot_type}:**'))
        # Create a TransferFunction object
        system = TransferFunction(num, den)

        if num_approx is not None and den_approx is not None:
            # Create TransferFunction object for approximated system
            system_approx = TransferFunction(num_approx, den_approx)

        if plot_type == 'Bode Plot':
            # Bode Plot
            w = np.logspace(-2, 2, 1000)
            w, mag, phase = bode(system, w=w)

            # Plotting
            fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(10, 8))

            # Magnitude plot
            ax_mag.semilogx(w, mag, label='Original')  # Bode magnitude plot
            ax_mag.set_title('Bode Plot')
            ax_mag.set_ylabel('Magnitude (dB)')
            ax_mag.grid(True, which='both', linestyle='--')

            # Phase plot
            ax_phase.semilogx(w, phase, label='Original')  # Bode phase plot
            ax_phase.set_xlabel('Frequency (rad/s)')
            ax_phase.set_ylabel('Phase (degrees)')
            ax_phase.grid(True, which='both', linestyle='--')

            if num_approx is not None and den_approx is not None:
                # Bode Plot of Approximated Function
                w_approx, mag_approx, phase_approx = bode(system_approx, w=w)

                # Plot the approximated magnitude and phase
                ax_mag.semilogx(w_approx, mag_approx, label='Approximated')
                ax_phase.semilogx(w_approx, phase_approx, label='Approximated')

                ax_mag.legend()
                ax_phase.legend()

            plt.tight_layout()
            plt.show()

        elif plot_type == 'Impulse Response':
            # Impulse Response
            t, y = impulse(system)
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(t, y, label='Original')
            ax.set_title('Impulse Response')
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Amplitude')
            ax.grid(True)

            if num_approx is not None and den_approx is not None:
                t_approx, y_approx = impulse(system_approx, T=t)
                ax.plot(t_approx, y_approx, label='Approximated')

            ax.legend()
            plt.show()

        elif plot_type == 'Step Response':
            # Step Response
            t, y = step(system)
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(t, y, label='Original')
            ax.set_title('Step Response')
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Amplitude')
            ax.grid(True)

            if num_approx is not None and den_approx is not None:
                t_approx, y_approx = step(system_approx, T=t)
                ax.plot(t_approx, y_approx, label='Approximated')

            ax.legend()
            plt.show()

def calculate_transfer_function(b):
    try:
        num_coeffs_list = [parse_complex(c) for c in num_coeffs_input.value.split(',') if c.strip()]
        den_coeffs_list = [parse_complex(c) for c in den_coeffs_input.value.split(',') if c.strip()]
        num_poles = num_poles_input.value
        plot_type = plot_type_widget.value
        
        num = np.array(num_coeffs_list, dtype=np.complex128)
        den = np.array(den_coeffs_list, dtype=np.complex128)
        
        zeros = np.roots(num)
        poles = np.roots(den)
        
        residues, poles_res, k_res = residue(num, den)
        
        # Perform dominant pole approximation
        num_approx, den_approx = dominant_pole_approximation(num, den, num_poles)
        
        display_transfer_function(num, den, zeros, poles, residues, poles_res, k_res, num_approx, den_approx, plot_type)
    except:
        pass



# Input widget for number of dominant poles
num_poles_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Number of Dominant Poles:',
    disabled=False
)

# Plot type selection
plot_type_widget = widgets.Select(
    options=['Impulse Response', 'Step Response', 'Bode Plot'],
    value='Step Response',
    description='Response Comparison:',
    disabled=False
)
# Input widgets for numerator and denominator coefficients
num_coeffs_input = widgets.Text(description='Numerator Coeffs:', placeholder='Enter coeffs, comma-separated')
den_coeffs_input = widgets.Text(description='Denominator Coeffs:', placeholder='Enter coeffs, comma-separated')
calc_button = widgets.Button(description='Calculate Transfer Function')

display( num_coeffs_input, den_coeffs_input, num_poles_input, plot_type_widget, calc_button)
calc_button.on_click(calculate_transfer_function)


### Constraints on Time-Domain Specifications

Above we have seen how to read the time-domain specifications directly off the time response. However, this is for the current system configuration -- i.e., given the transfer function of a system (open or closed-loop), we can read what the current time-domain specifications are. However, in many cases, we need to the design the system such that it achieves specified requirements. One way of doing this could be to parametrize the time response plots like above, and then determine suitable parameters for the system. However, since the time response of the systems is coupled to the poles of the systems, provided with the required time-domain specifications, it is possible to identify on the s-plane, the area in which the poles should be present.

In general how the area of satisfaction looks on the s-plane will vary highly according to the problem at hand -- the following identities may come into use.
$$
\begin{align}
    \omega_n &= \sqrt{\sigma^2 + \omega^2} \\
    |\sigma| &= \zeta \omega_n \\
    \varphi &= \arctan{\frac{\sigma}{\omega}} \\
    \zeta &= \sin{|\varphi|}
\end{align}
$$
Furthermore, in Problem Set 7 we have provided a couple questions that go through this process -- essentially reverse engineering the system parameters such that its poles are within the suitable area.

Nevertheless, it is often useful to gain an intuitive understanding as to how the time specifications affect area of satisfaction on the s-plane. Let's consider the following time-domain specifications:

- **Settling Time**: The settling time $T_d$ is directly linked to the real value of the poles. Recall $T_d = \tau \ln{\frac{100}{d}}$, where $\tau$ is determined off the real value of the poles. If $T_d$ must be smaller/greater than some value, then correspondingly, it is possible to identify the range of values for the poles. 

- **Peak Overshoot Ratio**: Recall that the peak overshoot ratio $M_p$ is directly linked to the damping ratio. Furthermore, since $|\varphi| = \arcsin(\zeta) = \left|\arctan{\frac{\sigma}{\omega}}\right|$, where $\varphi$ is the angle between the pole and the zero axis, if provided with a desired maximum/minimum overshoot, it is possible to identity a range of locations in which the pole should lie such that the overshoot requirement is met.

- **Rise Time**: Recall that the rise time is linked to the natural frequency $\omega_n$ for the system. Since $\omega_n = \sqrt{\sigma^2 + \omega^2}$, it is then possible to determine a range of values such that the poles lie in/out of the radius $\omega_n$, if the rise time should be more\less than some value.

#### Visualization


To help build the above intuition, we have provided an interactive visualization below for the three time-domain specifications. We have colored the regions that correspond with poles that results in $T_d, M_p, T_{100}$ being less than some specified value (e.g., the red region results in poles that satisfy the settling time requirement). In the case that any of the requirements should be more than some specified value, then the area of satisfaction is to be flipped. 

Try varying each of the requirements and see in which way the area of satisfaction moves.
Also, feel free to test yourself by setting the values for the requirements, and then determining the area in which the poles should lie such that the requirements are met. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def plot_pole_cone(max_overshoot, T_r, T_s):
    zeta = -np.log(max_overshoot / 100) / np.sqrt(np.pi**2 + (np.log(max_overshoot / 100))**2)
    theta = np.arcsin(zeta)
    cone_angle = theta
    cone_slope = 1 / np.tan(cone_angle)
    
    real_parts = np.linspace(-200, 0, 400)
    imag_upper = cone_slope * (-real_parts)
    imag_lower = -imag_upper

    # Calculate omega_n and sigma
    omega_n = (2 * np.pi / T_r)
    sigma = -np.log(0.02) / T_s

    plt.figure(figsize=(10, 10))

    # Region within cone angle
    plt.fill_between(real_parts, imag_upper, imag_lower, color='lightblue', alpha=0.3, label=fr'$\varphi \geq$ {theta * (180 / np.pi):.2f}°')

    # Circle for omega_n limit (showing the inside of the circle only)
    circle = plt.Circle((0, 0), omega_n, color='orange', fill=False, linestyle='--')
    plt.gca().add_artist(circle)

    # Left of sigma
    plt.fill_betweenx(np.linspace(-70, 70, 500), -200, -sigma, color='lightcoral', alpha=0.2, label=fr'$\sigma \leq$ -{sigma:.2f}')
    plt.axvline(x=-sigma, color='red', linestyle='--',)

    # Outside of omega_n (mask everything inside omega_n radius)
    theta_vals = np.linspace(0, 2 * np.pi, 500)
    outer_x = omega_n * np.cos(theta_vals)
    outer_y = omega_n * np.sin(theta_vals)
    plt.fill_betweenx(outer_y, 200, outer_x, where=(outer_x >= omega_n), color='orange', alpha=0.1, label=fr'$\sqrt{{\sigma^2 + \omega^2}} \geq$ {omega_n:.2f}')
    plt.fill_between(np.linspace(-200, -omega_n, 500), -70, 70, color='orange', alpha=0.1)
    plt.fill_between(np.linspace(omega_n, 200, 500), -70, 70, color='orange', alpha=0.1)
    
    # Mask inside the circle (region outside of omega_n)
    x_outer = np.linspace(-omega_n, omega_n, 500)
    y_outer = np.sqrt(omega_n**2 - x_outer**2)
    plt.fill_between(x_outer, y_outer, 70, color='orange', alpha=0.1)
    plt.fill_between(x_outer, -y_outer, -70, color='orange', alpha=0.1)

    # Draw axes
    plt.axhline(0, color='black', lw=0.5, ls='--')
    plt.axvline(0, color='black', lw=0.5, ls='--')

    plt.title(fr'Permissible Pole Locations ($M_p \leq$ {max_overshoot:.2f}%, $T_r \leq$ {T_r:.2f}s, $T_d \leq$ {T_s:.2f}s)')
    plt.xlim(-70, 70)
    plt.ylim(-70, 70)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid()
    plt.legend()
    plt.show()

# Define interactive sliders
max_overshoot_slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description='Max Overshoot (%)')
T_r_slider = widgets.FloatSlider(value=0.5, min=0.01, max=1, step=0.01, description='Rise Time (s)')
T_s_slider = widgets.FloatSlider(value=0.1, min=0.01, max=5, step=0.01, description='Settling Time (s)')

# Create interactive plot
interactive_plot = widgets.interactive(plot_pole_cone,
                                       max_overshoot=max_overshoot_slider,
                                       T_r=T_r_slider,
                                       T_s=T_s_slider)
display(interactive_plot)

# PID Control



**PID control** or *Proportional-Integral-Derivative* control is a very common type of control used in a large variety of use cases.

## What is a PID Controller?


A PID controller is a dynamic compensator that uses a combination of three gains: **Proportional (P)**, **Integral (I)**, and **Derivative (D)** with the aim of ensuring that the system output will arrive a desired setpoint meeting some system requirements. Intuitively, the controller continuously calculates an error value as the difference between a desired setpoint and a measured process variable, and applies correction based on proportional, integral, and derivative terms.

Let's break down each component to understand how they contribute to the control action.


## The Components of PID Control


### 1. Proportional Control (P)



**Concept:** The proportional term produces an output value that is proportional to the current error value.

**Mathematical Representation:**

$$
\text{Output}_P = K_P \times e(t)
$$

- $ K_P $ is the proportional gain.
- $ e(t) $ is the error at time $ t $.

**Intuitive Explanation:**

Imagine you're steering a car to stay in the center of a lane. If you're drifting to the left, you need to steer to the right to correct your position. The further you are from the center, the more you need to turn the wheel. This immediate response to the error is akin to proportional control.

**Characteristics:**

- **Advantages:**
  - Simple and provides immediate corrective action.
- **Limitations:**
  - May not eliminate steady-state error (a constant offset from the desired setpoint).
  - May induce further oscillations at high gains.




### 2. Integral Control (I)



**Concept:** The integral term accounts for the accumulation of past errors by integrating the error over time. This provides a corrective action to eliminate residual steady-state errors.

**Mathematical Representation:**

$$
\text{Output}_I = K_I \times \int_{0}^{t} e(\tau) \, d\tau
$$

- $ K_I $ is the integral gain.

**Intuitive Explanation:**

Continuing with the car analogy, if you've been slightly off-center for a while, even if the error is small, over time it adds up. The integral term increases the corrective action the longer the error persists, pushing you back to the center.

**Characteristics:**

- **Advantages:**
  - Eliminates steady-state error.
- **Limitations:**
  - Can introduce overshoot and oscillations if not tuned properly.




### 3. Derivative Control (D)



**Concept:** The derivative term accounts for (potential) future errors by calculating the rate of change of the error. It provides a damping action, reacting to how quickly the error is changing.

**Mathematical Representation:**

$$
\text{Output}_D = K_D \times \frac{de(t)}{dt}
$$

- $ K_D$ is the derivative gain.

**Intuitive Explanation:**

If you're correcting your car's position and you notice that the rate at which you're approaching the center is fast, you might ease off the steering to prevent overshooting. The derivative term helps anticipate future errors and adjusts the control action accordingly.

**Characteristics:**

- **Advantages:**
  - Improves system stability and response time.
- **Limitations:**
  - Sensitive to noise in the error signal.

## Combining the PID Terms



The PID controller combines all three terms to calculate the control output:

$$
\text{Control Output} = \text{Output}_P + \text{Output}_I + \text{Output}_D
$$

Or, in full:

$$
\text{Control Output} = K_P e(t) + K_I \int_{0}^{t} e(\tau) \, d\tau + K_D \frac{de(t)}{dt}
$$

With the corresponding transfer function:
$$
C(s) = K_P + \frac{K_I}{s} + K_Ds
$$

Each term addresses different aspects of the system's performance:

- **Proportional:** Addresses present error.
- **Integral:** Addresses accumulated past error.
- **Derivative:** Predicts future error.


## How Does a PID Controller Work?



1. **Measurement:** The system measures the current process variable (e.g., temperature, speed).

2. **Error Calculation:** It computes the error $ e(t) $ by subtracting the process variable from the desired setpoint.

3. **Control Action Calculation:** The controller calculates the control output using the PID formula.

4. **Actuation:** The control output is applied to the system through an actuator (e.g., adjusting a valve, changing motor voltage).

5. **Feedback Loop:** Steps 1-4 repeat continuously, allowing the system to respond to changes and disturbances.


## Tuning PID Controllers



Selecting the appropriate gains $ K_P, K_I$, and $K_D $ is crucial for optimal performance.

- **Under-Tuned Controller:**
  - Slow response.
  - May not reach the setpoint efficiently.

- **Over-Tuned Controller:**
  - Can cause excessive overshoot.
  - May lead to oscillations or instability.

**Common Tuning Methods:**

- **Manual Tuning:**
  - Adjust gains based on observation.

- **Ziegler-Nichols Method:**
  - An approach that uses specific rules.

- **Software Tools:**
  - Use algorithms to optimize gains, such as MATLABs Control System Designer.


## Root Locus based Tuning


Another approach to tuning the PID parameters, is by plotting the root locus to identify whether there exists some gain $K$, such that the system meets the design specifications. 

First recall that the root locus plot consists of the closed loop poles of a system as a function of some parameter. Although in principle we can vary any parameter, tools such as MATLAB and Python, assume the standard feedback architecture we have previously seen -- i.e., provided with an open-loop transfer function $L(s)$, they compute the closed-loop poles of $T(s)= \frac{KL(s)}{1+KL(s)}$. Thus, when using such tools to fine-tune controllers, it is important to keep in mind when plotting the root locus, which is the plotted gain. 

Two possible approaches to integrating a PID controller with the root locus are too:
1. Set $C(s) = K_P \left( 1 + \frac{T_I}{s} + T_D s\right)$, where $K_P T_I = K_I$ and $K_P T_D = K_D$. Then the closed loop transfer function is of the form $T(s) = \frac{K_P \left( 1 + \frac{T_I}{s} + T_D s\right)L(s)}{1+K_P\left( 1 + \frac{T_I}{s} + T_D s\right)P(s)}$. You then set fixed values of $T_I$ and $T_D$, and if there exists some $K_P$ that results in the closed-loop poles being in the ``good'' region of the s-plane, then you can calculate the corresponding $K_P, K_I, K_D$ for your controller.  
2. Set $C(s) = K_P + \frac{K_I}{s} + K_D s$. Then set $K_P, K_I$ and $K_D$ to some fixed values. Then plot the root locus for the closed-loop system $T(s)= \frac{kL(s)}{1+kL(s)}$. Then, by looking at the root locus, for the values of $K$ for which the system satisfies your requirements, the values for the PID controller are multiplied by $K$.

Below we have provided a tool that supports both options -- i.e., the first approach is a case of the second where $K=1$ --, feel free to test and ensure that both return the same "final" values of the controller.

# PS07 Problems


To support with the tasks in the problem sheet, below we have provided a tool. You can input a transfer function, the maximum value of the system requirements, and then design a PID controller (using either of the 2 approaches above) such that the system satisfies your requirements.

*Note*: The plotted requirements are valid for systems in the standard forms provided in the lecture. If the open-loop transfer function contains zeros, the corresponding formulations for the requirements changes, and thus so does the corresponding "good" region in the s-plane. Nevertheless, we have left the colored regions to aid an intuitive understanding of the process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
import ipywidgets as widgets
from IPython.display import display

# Define a function to simulate the closed-loop response with a PID controller
def pid_response(num_str, den_str, kp, ki, kd, overshoot_requirement, settling_time_requirement, rise_time_requirement, K, Tmax):

    zeta = -np.log(overshoot_requirement / 100) / np.sqrt(np.pi**2 + (np.log(overshoot_requirement / 100))**2)
    theta = np.arcsin(zeta)
    cone_angle = theta
    cone_slope = 1 / np.tan(cone_angle)
    
    real_parts = np.linspace(-1000, 0, 400)
    imag_upper = cone_slope * (-real_parts)
    imag_lower = -imag_upper

    # Calculate omega_n and sigma
    omega_n = (0.14 + 0.4 * zeta )*(2 * np.pi / rise_time_requirement)
    sigma = -np.log(0.02) / settling_time_requirement

    num = [float(x) for x in num_str.split(',')]
    den = [float(x) for x in den_str.split(',')]
    plant = ctrl.TransferFunction(num, den)

    # Define the PID controller transfer function
    if ki == 0:
        pid_num = [kd, kp]
        pid_den = [1]
    else:
        pid_num = [kd, kp, ki]
        pid_den = [1, 0]
    pid = ctrl.TransferFunction(pid_num, pid_den)

    # Combine PID and plant in series by manually multiplying the numerators and denominators
    system_num = np.polymul(pid.num[0][0], plant.num[0][0])
    system_den = np.polymul(pid.den[0][0], plant.den[0][0])

    # Open-loop transfer function
    open_loop = ctrl.TransferFunction(system_num, system_den)

    # Create the closed-loop system with feedback gain K
    closed_loop = ctrl.feedback(K * open_loop)

    # Create the figure and subplots
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    
    # Root locus plot in the first subplot
    axs[0].set_title("Root Locus")
    axs[0].set_xlabel("Real Axis")
    axs[0].set_ylabel("Imaginary Axis")
    rlist, klist = ctrl.root_locus(open_loop, plot=False)
    poles_cl = closed_loop.poles()
    axs[0].scatter(poles_cl.real, poles_cl.imag, marker='x', color='r', label='Closed Loop Poles')
    axs[0].plot(rlist.real, rlist.imag, 'b', label="Root Locus")
    axs[0].grid(True)
    axs[0].set_ylim(min(-10, -omega_n), max(10, omega_n))
    axs[0].set_xlim(min(min(-sigma-2, -omega_n), max(min(poles_cl.real),-1000)), max(10, omega_n))

    # Region within cone angle
    axs[0].fill_between(real_parts, imag_upper, imag_lower, color='lightblue', alpha=0.3, label=fr'$\varphi \geq$ {theta * (180 / np.pi):.2f}°')

    # Circle for omega_n limit (showing the inside of the circle only)
    circle = plt.Circle((0, 0), omega_n, color='orange', fill=False, linestyle='--')
    axs[0].add_artist(circle)

    # Left of sigma
    axs[0].fill_betweenx(np.linspace(-70, 70, 500), -1000, -sigma, color='lightcoral', alpha=0.2, label=fr'$\sigma \leq$ -{sigma:.2f}')
    axs[0].axvline(x=-sigma, color='red', linestyle='--',)

    # Outside of omega_n (mask everything inside omega_n radius)
    theta_vals = np.linspace(0, 2 * np.pi, 500)
    outer_x = omega_n * np.cos(theta_vals)
    outer_y = omega_n * np.sin(theta_vals)
    axs[0].fill_betweenx(outer_y, 200, outer_x, where=(outer_x >= omega_n), color='orange', alpha=0.1, label=fr'$\sqrt{{\sigma^2 + \omega^2}} \geq$ {omega_n:.2f}')
    axs[0].fill_between(np.linspace(-1000, -omega_n, 500), -70, 70, color='orange', alpha=0.1)
    axs[0].fill_between(np.linspace(omega_n, 200, 500), -70, 70, color='orange', alpha=0.1)
    
    # Mask inside the circle (region outside of omega_n)
    x_outer = np.linspace(-omega_n, omega_n, 500)
    y_outer = np.sqrt(omega_n**2 - x_outer**2)
    axs[0].fill_between(x_outer, y_outer, 70, color='orange', alpha=0.1)
    axs[0].fill_between(x_outer, -y_outer, -70, color='orange', alpha=0.1)

    # # Draw axes
    axs[0].axhline(0, color='black', lw=0.5, ls='--')
    axs[0].axvline(0, color='black', lw=0.5, ls='--')

    # Step response plot in the second subplot
    t, y = ctrl.step_response(closed_loop, T=np.linspace(0, Tmax, 500))
    axs[1].plot(t, y, label="Step Response")
    
    # Plot performance requirements on the step response plot
    axs[1].axhline(1 + overshoot_requirement/100, color='r', linestyle='--', label=f'Overshoot: {overshoot_requirement}%')
    axs[1].axhline(1 - overshoot_requirement/100, color='r', linestyle='--')
    axs[1].axvline(settling_time_requirement, color='g', linestyle='--', label=f'Settling Time: {settling_time_requirement}s')
    axs[1].axvline(rise_time_requirement, color='b', linestyle='--', label=f'Rise Time: {rise_time_requirement}s')
    
    axs[1].set_title(f'Step Response with PID (kp={kp:.2f}, ki={ki:.2f}, kd={kd:.2f})')
    axs[1].set_xlabel('Time [s]')
    axs[1].set_ylabel('Output y(t)')
    axs[1].legend(loc='best')
    axs[1].grid(True)
    axs[1].set_ylim(0, 1.2)
    axs[1].set_xlim(0, max(t))
    
    plt.tight_layout()
    plt.show()

# Input widgets
plant_num_input = widgets.Text(
    description='Plant Numerator:',
    placeholder='72.25',
    value='1'
)
plant_den_input = widgets.Text(
    description='Plant Denominator:',
    placeholder='Enter coeffs, highest power first',
    value='1, 3, 10'
)

# Input widgets
overshoot_req_widget = widgets.FloatSlider(value=5, min=0, max=100, step=0.1, description='Overshoot [%]:')
settling_time_req_widget = widgets.FloatSlider(value=0.1, min=0, max=10, step=0.01, description='Settling Time [s]:')
rise_time_req_widget = widgets.FloatSlider(value=0.05, min=0, max=10, step=0.01, description='Rise Time [s]:')


# Create sliders for PID parameters
K_p_slider = widgets.FloatSlider(value=1.0, min=0, max=1000.0, step=1, description='K_p:')
K_i_slider = widgets.FloatSlider(value=0.0, min=0, max=500.0, step=1, description='K_i:')
K_d_slider = widgets.FloatSlider(value=2.0, min=0, max=500.0, step=1, description='K_d:')
K_slider = widgets.FloatSlider(value=87.0, min=0, max=500.0, step=1, description='K:')

# time axis slider 
t_max_slider = widgets.FloatSlider(value=8, min=0, max=10, step=0.01, description='Max Time [s]:')

# Output widget for plot
output = widgets.interactive_output(pid_response, {
    'num_str':plant_num_input,
    'den_str':plant_den_input,
    'kp': K_p_slider, 'ki': K_i_slider, 'kd': K_d_slider,
    'overshoot_requirement': overshoot_req_widget,
    'settling_time_requirement': settling_time_req_widget,
    'rise_time_requirement': rise_time_req_widget,
    'K': K_slider,
    'Tmax': t_max_slider})

# Group the widgets into columns
left_column = widgets.VBox([plant_num_input, plant_den_input, overshoot_req_widget, settling_time_req_widget, rise_time_req_widget])
middle_column = widgets.VBox([K_p_slider, K_i_slider, K_d_slider, K_slider])
right_column = widgets.VBox([t_max_slider])
# Arrange the two columns side by side
input_layout = widgets.HBox([left_column, middle_column, right_column])

display(input_layout, output)